# Evaluation Summary Plots/Anaylisis

Plots/Analysis for a single evaluation for a single experiment. For now, only the default evaluation is considered.

In [ ]:
# Common imports.
from pathlib import Path

%matplotlib ipympl
import base

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import dfaas_env
import dfaas_utils

Global options for the notebook.

* `exp_dir`: the full path for the experiment directory.

In [ ]:
# TODO: Make this configurable.
exp_dir = Path("/home/emanuele/marl-dfaas/results/DFAAS-MA_2024-12-20_11-49-29_500_SYN")

## Reward (all episodes)

In [ ]:
# Common functions for average reward data.


def _get_data(eval_dir):
    data = {}

    # Read data from the given evaluation directory
    eval = dfaas_utils.parse_result_file(eval_dir / "evaluation.json")
    eval = eval[0]["env_runners"]
    agents = base.get_env(eval_dir).agents

    data["agents"] = agents
    data["episodes"] = eval["episodes_this_iter"]

    reward_total = {}  # Total reward per episode.
    reward_total["all"] = eval["hist_stats"]["episode_reward"]
    for agent in data["agents"]:
        reward_total[agent] = eval["hist_stats"][f"policy_policy_{agent}_reward"]

    data["reward_total_avg"] = reward_total

    return data

Get `data` and `env` variables, used by subsequents plots.

In [ ]:
data = _get_data(exp_dir)
env = base.get_env(exp_dir)

### Average reward per episode (all agents)

In [ ]:
fig = plt.figure(layout="constrained")
fig.canvas.header_visible = False
ax = fig.subplots()

# Limits for the y axis, both for total and single step.
bottom, top = env.reward_range
bottom_total = bottom * env.max_steps
top_total = top * env.max_steps

# First line: theoretical limit of reward.
ax.plot(np.full(data["episodes"], top_total * len(data["agents"])), color="r", label="Limit")

# Second line: real reward.
ax.plot(data["reward_total_avg"]["all"], label="Reward")
ax.set_ylim(bottom=bottom_total, top=top_total * len(data["agents"])+10)
ax.set_title("Average reward per episode (all agents)")
ax.yaxis.set_major_locator(ticker.MultipleLocator(50))

ax.set_ylabel("Reward per episode")

ax.set_xlabel("Episode")
ax.xaxis.set_major_locator(
    ticker.MultipleLocator(10)
)  # Show x-axis ticks every 10 episodes.

ax.grid(axis="both")
ax.set_axisbelow(True)  # By default the axis is over the content.
ax.legend()

### Average reward per episode (single agents)

In [ ]:
for agent in data["agents"]:
    fig = plt.figure(layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()
    
    # Limits for the y axis, both for total and single step.
    bottom, top = env.reward_range
    bottom_total = bottom * env.max_steps
    top_total = top * env.max_steps

    # First line: theoretical limit of reward.
    ax.plot(np.full(data["episodes"], top_total), color="r", label="Limit")
    
    # Second line: real reward.
    ax.plot(data["reward_total_avg"][agent], label="Reward")
    ax.set_ylim(bottom=bottom_total, top=top_total+10)
    ax.set_title(f"Average reward per episode ({agent = })")
    ax.yaxis.set_major_locator(ticker.MultipleLocator(50))
    
    ax.set_ylabel("Reward per episode")
    
    ax.set_xlabel("Episode")
    ax.xaxis.set_major_locator(
        ticker.MultipleLocator(10)
    )  # Show x-axis ticks every 10 episodes.
    
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.
    ax.legend()

## Queue size (single episode)

In [ ]:
# Common functions to get data.

def _get_data(eval_dir):
    # Read data from the given evaluation directory
    eval = dfaas_utils.parse_result_file(eval_dir / "evaluation.json")
    eval = eval[0]["env_runners"]
    agents = base.get_env(eval_dir).agents

    data = {}
    data["agents"] = agents
    data["queue_size"] = eval["hist_stats"]["queue_size"]

    return data

In [ ]:
# Get data and env variables, used by the subsequent code.
data = _get_data(exp_dir)
env = base.get_env(exp_dir)
episode_idx = 0  # Which episode to show in the plots.

In [ ]:
for agent in data["agents"]:
    fig = plt.figure(layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    ax.set_title(f"Queue status for {agent = } and {episode_idx = }")
    ax.plot(data["queue_size"][episode_idx][agent], label="Requests in queue")
    ax.plot(np.full(env.max_steps, env.queue_capacity), color="r", label="Queue capacity")
    
    ax.set_ylim(bottom=bottom, top=env.queue_capacity+10)  # Limits for the y axis.
    ax.yaxis.set_major_locator(ticker.MultipleLocator(50))
    
    ax.set_ylabel("Requests")
    
    ax.set_xlabel("Step")
    ax.xaxis.set_major_locator(
        ticker.MultipleLocator(25)
    )  # Show x-axis ticks every 25 steps.
    
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))  # Show legend outside the plot at the right.

## Processed requests (single episode)

In [ ]:
def _get_data(eval_dir, episode_idx):
    # Read data from the given evaluation directory
    eval = dfaas_utils.parse_result_file(eval_dir / "evaluation.json")
    eval = eval[0]["env_runners"]
    agents = base.get_env(eval_dir).agents

    data = {}
    data["agents"] = agents

    # We must convert list to numpy arrays to allow array manipulation.
    data["processed_local"] = {}
    data["processed_local_forward"] = {}
    for agent in agents:
        data["processed_local"][agent] = np.array(eval["hist_stats"]["processed_local"][episode_idx][agent])
        data["processed_local_forward"][agent] = np.array(eval["hist_stats"]["processed_local_forward"][episode_idx][agent])
    
    return data

In [ ]:
# Get data and env variables, used by the subsequent code.
episode_idx = 0  # Which episode to show in the plots.
data = _get_data(exp_dir, episode_idx)
env = base.get_env(exp_dir)

In [ ]:
for agent in data["agents"]:
    fig = plt.figure(layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    steps_x = np.arange(stop=env.max_steps)

    ax.set_title(f"Processed requests for {agent = } and {episode_idx = }")
    ax.bar(x=steps_x,
           height=data["processed_local"][agent] - data["processed_local_forward"][agent],
           color="g",
           label="Processed requests (local)")
    ax.bar(x=steps_x,
           height=data["processed_local_forward"][agent],
           bottom=data["processed_local"][agent] - data["processed_local_forward"][agent],
           color="b",
           label="Processed requests (forwarded)")
    
    ax.set_ylabel("Requests")
    
    ax.set_xlabel("Step")
    ax.xaxis.set_major_locator(
        ticker.MultipleLocator(25)
    )  # Show x-axis ticks every 25 steps.
    
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.
    ax.legend()

## Action (single episode)

In [ ]:
def _get_data(eval_dir, episode_idx):
    # Read data from the given evaluation directory
    eval = dfaas_utils.parse_result_file(eval_dir / "evaluation.json")
    eval = eval[0]["env_runners"]
    agents = base.get_env(eval_dir).agents

    data = {}
    data["agents"] = agents

    # We must convert list to numpy arrays to allow array manipulation.
    data["action_local"] = {}
    data["action_forward"] = {}
    data["action_reject"] = {}
    for agent in agents:
        data["action_local"][agent] = np.array(eval["hist_stats"]["action_local"][episode_idx][agent])
        data["action_forward"][agent] = np.array(eval["hist_stats"]["action_forward"][episode_idx][agent])
        data["action_reject"][agent] = np.array(eval["hist_stats"]["action_reject"][episode_idx][agent])
        
    return data

In [ ]:
# Get data and env variables, used by the subsequent code.
episode_idx = 0  # Which episode to show in the plots.
data = _get_data(exp_dir, episode_idx)
env = base.get_env(exp_dir)

In [ ]:
for agent in data["agents"]:
    fig = plt.figure(layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    steps_x = np.arange(stop=env.max_steps)

    ax.set_title(f"Actions for {agent = } and {episode_idx = }")
    ax.bar(x=steps_x,
           height=data["action_local"][agent],
           color="g",
           label="Local")
    ax.bar(x=steps_x,
           height=data["action_forward"][agent],
           bottom=data["action_local"][agent],
           color="b",
           label="Forward")
    ax.bar(x=steps_x,
           height=data["action_reject"][agent],
           bottom=data["action_local"][agent] + data["action_forward"][agent],
           color="r",
           label="Reject")

    input_requests = data["action_local"][agent] + data["action_forward"][agent] + data["action_reject"][agent]
    ax.plot(input_requests, label="Input requests")
    
    ax.set_ylabel("Requests")
    
    ax.set_xlabel("Step")
    ax.xaxis.set_major_locator(
        ticker.MultipleLocator(25)
    )  # Show x-axis ticks every 25 steps.
    
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.
    ax.legend()